In [1]:
import os

os.environ["LLAMA_CLOUD_API_KEY"] = "llx-QlZUAXGgDpfavBR40UJp6tvfH9h0fEsvTVk0oR9JzNi5bU9c"

### Load data

In [2]:
DATA_DIR = 'data/resumes'

In [3]:
fnames = os.listdir(DATA_DIR)
fnames = [fname for fname in fnames if fname.endswith('.pdf')]
fpaths = [os.path.join(DATA_DIR, fname) for fname in fnames]
fpaths

['data/resumes/14224370.pdf',
 'data/resumes/12780508.pdf',
 'data/resumes/19545827.pdf']

### Define a Pydantic Model

In [4]:
from pydantic import BaseModel

In [5]:
class Education(BaseModel):
    degree: str
    honors: str
    institution: str
    field_of_study: str
    graudation_year: str
    
class Resume(BaseModel):
    education: Education
    summary: str

### Create schema

In [6]:
from llama_extract import LlamaExtract

extractor = LlamaExtract(verbose=True)

In [7]:
schema_response = await extractor.acreate_schema('Resume Schema', data_schema=Resume)

### Run extraction

In [10]:
responses, models = await extractor.aextract(schema_response.id, fpaths, response_model=Resume)

Extracting files: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:06<00:00,  2.22s/it]


In [30]:
for model in models:
    print('=====')
    print('Summary:\t', model.summary)
    print('Institution:\t', model.education.institution)

=====
Summary:	 Degreed accountant with more than 10 years of diversified accounting experience seeking accounting position at a well-established company in Houston
Institution:	 University of Houston
=====
Summary:	 Provided customers with prompt, accurate, courteous and professional banking service. Identified and referred sales opportunities to Relationship Bankers about products and services. Utilized several mediums such as phone and emails to help customers. Assisted customers with opening and closing of accounts. Answered and resolved problems that are within my authority. Accepted and processed loan applications and conduct loan interviews. Assisted members with their financial transactions, involving paying and receiving cash and other negotiable instruments. Maintained proper cash levels at the branch. Responsible for cash shipments to and from main office to the branch. Processed all commercial deposits, balanced vault daily. Responsible for equipment maintenance; assisted s